In [1]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
import seaborn as sns
sns.set(color_codes=True)
import geopandas as gp
from scipy import stats, integrate

In [2]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOW_SpeedLimit.csv', encoding='utf-8')
herepath.head()

,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,SPDLIMIT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT_5,PCT_50,PCT_85,DOW
0,998969850F,2014-01-01 13:55,167,186,72,61.0,0.0,61,61,10,61,61,61,Wednesday
1,998969850F,2014-01-02 11:05,133,186,72,60.0,0.0,60,60,10,60,60,60,Thursday
2,998969850F,2014-01-02 13:20,160,186,72,61.0,0.0,61,61,10,61,61,61,Thursday
3,998969850F,2014-01-02 14:30,174,186,72,52.0,0.0,52,52,10,52,52,52,Thursday
4,998969850F,2014-01-03 09:05,109,186,72,57.0,0.0,57,57,10,57,57,57,Friday


In [3]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [4]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [5]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [6]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe(w,x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        link = v[v['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        link = x[x['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [49]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [50]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsBase(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y = pd.DataFrame(columns=[])
    
    y['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thAll'] = epoct
    y['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
   
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
      

    
    y = np.round(y,3)
    return z

In [66]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsALL(x):
    y = pd.DataFrame(columns=[])
    
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thAll'] = epoct
    y['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
    y = np.round(y,3)
    return y

In [57]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWd(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)

    
    y['PcntSpeed_85thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWd'] = epoct
    y['PcntInterval_85thWd'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWd'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [58]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWend(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thWend'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWend'] = epoct
    y['PcntInterval_85thWend'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWend'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [59]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdtime(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_60thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_85thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_Wdtime'] = epoct
    y['PcntInterval_Wdtime'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_Wdtime'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [60]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayAM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayAM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayAM'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayAM'] = epoct
    y['PcntInterval_WdayAM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayAM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [61]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayMD(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayMD'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayMD'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayMD'] = epoct
    y['PcntInterval_WdayMD'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayMD'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [62]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayPM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayPM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayPM'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayPM'] = epoct
    y['PcntInterval_WdayPM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayPM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [63]:
def Indeces(y):
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    y = np.round(y,3)
    return y

In [16]:
#For all days and all time periods
#bylink = HEREstatsALL(herepath)

#bylink.head()

In [17]:
#for weekdays
Wkd = timeframe('998969850F',herepath,0,288,True,Weekday)
SpeedWd = HEREstatsWd(Wkd)
SpeedWd.head()

,PcntSpeed_85thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,
998969850F,42.87,2799,3.826,9.22


In [29]:
#for weekdays all links
Wkd = timeframe2(herepath,0,288,True,Weekday)
SpeedWd = HEREstatsWd(Wkd)
SpeedWd.head()

,PcntSpeed_85thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,
107030432T,47.840,5023,6.682,10.344
107030439T,44.734,6775,9.013,9.937
107030440T,45.355,6339,8.433,10.262
107030444F,47.219,8045,10.703,11.301
107030445F,47.840,8008,10.653,11.282


In [18]:
#for weekends
Wend = timeframe('998969850F',herepath,0,288,True,Weekend)
SpeedWend = HEREstatsWend(Wend)
SpeedWend.head()

,PcntSpeed_85thWend,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend
LINK_DIR,,,,
998969850F,46.598,330,1.157,10.751


In [19]:
#for weekends all links
Wend = timeframe2(herepath,0,288,True,Weekend)
SpeedWend = HEREstatsWend(Wend)
SpeedWend.head()

,PcntSpeed_85thWend,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend
LINK_DIR,,,,
107030432T,50.325,723,2.414,10.988
107030439T,45.355,1082,3.612,9.874
107030440T,45.355,986,3.292,10.532
107030444F,47.840,1266,4.227,10.461
107030445F,49.083,1248,4.167,10.760


In [20]:
#Weekday Between 6AM and 8PM
Wdtime = timeframe('998969850F',herepath,72,240,True,Weekday)
SpeedWdtime = HEREstatsWdtime(Wdtime)
SpeedWdtime.head()

,PcntSpeed_60thWdtime,PcntSpeed_85thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime
LINK_DIR,,,,,
998969850F,36.035,41.627,2587,3.536,8.938


In [21]:
#Weekday Between 6AM and 8PM all links
Wdtime = timeframe2(herepath,72,240,True,Weekday)
SpeedWdtime = HEREstatsWdtime(Wdtime)
SpeedWdtime.head()

,PcntSpeed_60thWdtime,PcntSpeed_85thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime
LINK_DIR,,,,,
107030432T,41.627,47.840,4458,5.931,10.412
107030439T,38.521,44.734,6233,8.292,9.992
107030440T,37.899,45.355,5891,7.837,10.269
107030444F,40.198,47.219,7538,10.028,11.354
107030445F,40.384,47.219,7355,9.785,11.290


In [26]:
#Stats between 6AM and 9AM
WdtimeAM = timeframe('998969850F',herepath,72,108,True,Weekday)
SpeedWdayAM = HEREstatsWdayAM(WdtimeAM)
SpeedWdayAM.head()

,PcntSpeed_5thWdayAM,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM
LINK_DIR,,,,,
998969850F,18.018,33.838,440,0.728,8.698


In [16]:
#Stats between 6AM and 9AM all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
SpeedWdayAM = HEREstatsWdayAM(WdtimeAM)
SpeedWdayAM.head()

261


134

In [27]:
#Stats between 9AM and 3PM
WdtimeMD = timeframe('998969850F',herepath,108,180,True,Weekday)
SpeedWdayMD = HEREstatsWdayMD(WdtimeMD)
SpeedWdayMD.head()

,PcntSpeed_5thWdayMD,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD
LINK_DIR,,,,,
998969850F,18.639,34.014,1367,1.899,8.661


In [15]:
#Stats between 9AM and 3PM all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
SpeedWdayMD = HEREstatsWdayMD(WdtimeMD)
SpeedWdayMD.head()

261


134

In [28]:
#Stats between 3PM and 6PM
WdtimePM = timeframe('998969850F',herepath,180,216,True,Weekday)
SpeedWdayPM = HEREstatsWdayPM(WdtimeMD)
SpeedWdayPM.head()

,PcntSpeed_5thWdayPM,AvgSpeed_WdayPM,TotalIntervals_WdayPM,PcntInterval_WdayPM,StdDev_WdayPM
LINK_DIR,,,,,
998969850F,18.639,34.014,1367,1.899,8.661


In [14]:
#Stats between 3PM and 6PM all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
SpeedWdayPM = HEREstatsWdayPM(WdtimeMD)
SpeedWdayPM.head()

261


134

In [65]:
AllAM = timeframe('998969850F',herepath,72,108,False,Both)
HstatsAM = HEREstatsALL(AllAM)
HstatsAM.head()
IndecesAllAM = Indeces(HstatsAM)
IndecesAllAM.head()


KeyError: '5spd'

In [20]:
IndecesAllAM.head()

NameError: name 'IndecesAllAM' is not defined

In [21]:
IndecesAllMD.head()

NameError: name 'IndecesAllMD' is not defined

In [22]:
IndecesAllPM.head()

NameError: name 'IndecesAllPM' is not defined